In [1]:
import os
import sys

if "ORIGINAL_WORKING_DIRECTORY" not in globals():
    ORIGINAL_WORKING_DIRECTORY = os.getcwd()

parent_directory = os.path.dirname(ORIGINAL_WORKING_DIRECTORY)
os.chdir(parent_directory)
sys.path.insert(0, os.getcwd())

import pandas as pd
import numpy as np
from pandasql import sqldf

from modules.data_processing.data_processor import ExperimentProcessor
from modules.ab_testing.ab_test_manager import ABTestManager
from modules.data_processing.data_loader import get_all_data

### Data processing

In [2]:
data = get_all_data()
processor = ExperimentProcessor(data)
labeled_df = processor.label_experiments()
labeled_df.head()

,event_name,experiment_name,variant_id,user_id,purchases,attempts,with_purchase
0,CHECKOUT_1,buyingflow/escWebMLA,2874,59,0,1,False
1,CHECKOUT_1,buyingflow/escWebMLA,2874,311,1,1,True
2,CHECKOUT_1,buyingflow/escWebMLA,2874,466,1,3,True
3,CHECKOUT_1,buyingflow/escWebMLA,2874,475,0,5,False
4,CHECKOUT_1,buyingflow/escWebMLA,2874,773,1,1,True


### Evaluate Experiment

In [3]:

def evaluate_experiments(data, exclude_variants: str = None):
    if exclude_variants is not None:
        experiments_df = data[~data["variant_id"].isin(exclude_variants)].copy()
    else:
        experiments_df = data.copy()

    experiments = experiments_df.experiment_name.unique().tolist()
    analysis_list = []
    for e in experiments:
        df = experiments_df[experiments_df["experiment_name"] == e].copy()
        processor = ABTestManager(df)
        checks, tests = processor.run_analysis()
        test = tests.get("tests")
        if test is None:
            test_name = None
            p_value = None
            significance = None
            size_adequacy = None
        else:
            chi2 = test.get("chi_square")
            z_test = test.get("z-test")
            if chi2 is None:
                test_name = "z-test"
                p_value = z_test.get("p_value")
                significance = z_test.get("significant_difference")
            else:
                test_name = "chi_square"
                p_value = chi2.get("p_value")
                significance = chi2.get("significant_difference")

            variants_size_adequacy = checks["sample_size_adequacy"]
            size_adequacy = all(variants_size_adequacy.values())
        

        dict_ab = {
            "experiment_name":e,
            "num_of_variants":checks.get("num_of_variants"),
            "winner_id":tests.get("winner"),
            "p-value":p_value,
            "test":test_name,
            "significance": significance,
            "size_adequacy": size_adequacy,
        }
        analysis_list.append(dict_ab)
    return pd.DataFrame(analysis_list)

In [5]:
test_with_default = evaluate_experiments(labeled_df)
test_with_default

,experiment_name,num_of_variants,winner_id,p-value,test,significance,size_adequacy
0,buyingflow/escWebMLA,1,2874,NaN,None,None,None
1,buyingflow/user-track,1,6796,NaN,None,None,None
2,buyingflow/address_hub,1,3574,NaN,None,None,None
3,cookiesConsentBanner,2,DEFAULT,0.387338,z-test,False,False
4,frontend/assetsCdnDomainMLA,1,DEFAULT,NaN,None,None,None
5,mshops/HideTransitionModal,4,6646,0.351113,chi_square,False,False
6,pdp/compatibilityWidget,1,5539,NaN,None,None,None
7,pdp/cpgShowOnlyAddToCart,1,6690,NaN,None,None,None
8,pdp/viewItemPageMigrationDesktopHirableSRV,2,7174,1.000000,z-test,False,False
9,pdp/viewItemPageMigrationDesktopQuotableSRV,2,7178,1.000000,z-test,False,False


### Evaluate remove DEFAULT variant

In [34]:
test_without_default = evaluate_experiments(labeled_df, ["DEFAULT"])

merge_df = test_with_default.merge(
    test_without_default,
    how="left",
    on = "experiment_name",
    suffixes=["", "_no_default"]
)
merge_df.head()

,experiment_name,num_of_variants,winner_id,p-value,test,significance,size_adequacy,num_of_variants_no_default,winner_id_no_default,p-value_no_default,test_no_default,significance_no_default,size_adequacy_no_default
0,buyingflow/escWebMLA,1,2874,NaN,None,None,None,1.0,2874,NaN,None,None,None
1,buyingflow/user-track,1,6796,NaN,None,None,None,1.0,6796,NaN,None,None,None
2,buyingflow/address_hub,1,3574,NaN,None,None,None,1.0,3574,NaN,None,None,None
3,cookiesConsentBanner,2,DEFAULT,0.387338,z-test,False,False,1.0,6686,NaN,None,None,None
4,frontend/assetsCdnDomainMLA,1,DEFAULT,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN


### Num of variants changed

In [35]:
merge_df["num_variants_changed"] = merge_df["num_of_variants"] != merge_df["num_of_variants_no_default"]

In [36]:
merge_df.num_variants_changed.value_counts()

num_variants_changed
False    32
True     14
Name: count, dtype: int64

In [37]:
merge_df["size_adequacy_no_default"] = np.where(
    merge_df["size_adequacy_no_default"].isna(),
    False,
    merge_df["size_adequacy_no_default"]
)

In [40]:
merge_df[merge_df["num_variants_changed"]].groupby(["size_adequacy", "size_adequacy_no_default"]).size().reset_index()

,size_adequacy,size_adequacy_no_default,0
0,False,False,9
1,False,True,2


### Winner id changed

In [49]:
merge_df["winner_id_changed"] = merge_df["winner_id"] != merge_df["winner_id_no_default"]

In [50]:
merge_df[(merge_df["num_of_variants"]>1) & (merge_df["num_of_variants_no_default"] > 1)]["winner_id_changed"].value_counts()

winner_id_changed
False    16
Name: count, dtype: int64

### Significance Test

In [54]:
merge_df["significance_no_default"] = np.where(
    merge_df["significance_no_default"].isna(),
    False,
    merge_df["significance_no_default"]
)

In [57]:
merge_df.groupby(["significance", "size_adequacy","significance_no_default"]).size().reset_index()

,significance,size_adequacy,significance_no_default,0
0,False,False,False,15
1,False,True,False,3
2,True,False,False,4
3,True,True,True,2


### Evaluate Exeperiments without DEFAULT variant

In [4]:
no_default_variant = labeled_df[~labeled_df["variant_id"].isin(["DEFAULT"])]

evaluate_experiments(no_default_variant)

,experiment_name,num_of_variants,winner_id,p-value,test,significance,size_adequacy
0,buyingflow/escWebMLA,1,2874,NaN,None,None,None
1,buyingflow/user-track,1,6796,NaN,None,None,None
2,buyingflow/address_hub,1,3574,NaN,None,None,None
3,mshops/HideTransitionModal,3,6646,0.598250,chi_square,False,False
4,pdp/compatibilityWidget,1,5539,NaN,None,None,None
5,pdp/cpgShowOnlyAddToCart,1,6690,NaN,None,None,None
6,pdp/viewItemPageMigrationDesktopHirableSRV,2,7174,1.000000,z-test,False,False
7,pdp/viewItemPageMigrationDesktopQuotableSRV,2,7178,1.000000,z-test,False,False
8,pdp/viewItemPageMigrationDesktopRES,1,5941,NaN,None,None,None
9,pdp/viewItemPageMigrationDesktopRESDev,1,6861,NaN,None,None,None
